In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, max, sum, row_number, month, lower
from pyspark.sql.window import Window

# Initialize Spark session (if not already done)
spark = SparkSession.builder.appName("CovidDataAnalysis").getOrCreate()


In [3]:
# Load CSV file
df = spark.read.csv('complete.csv', header=True, inferSchema=True)
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Name of State / UT: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Total Confirmed cases: double (nullable = true)
 |-- Death: string (nullable = true)
 |-- Cured/Discharged/Migrated: double (nullable = true)
 |-- New cases: integer (nullable = true)
 |-- New deaths: integer (nullable = true)
 |-- New recovered: integer (nullable = true)



In [10]:
# 1. Convert all state names to lowercase
df = df.withColumn('Name of State / UT', lower(col('Name of State / UT')))

# 2. The day with the maximum number of COVID-19 cases
# day_with_max_cases = df.groupBy('Date').agg(max('Total Confirmed cases').alias('max_cases'))
# day_with_max_cases = day_with_max_cases.orderBy(col('max_cases').desc())
# # day_with_max_cases.show() 
# max_day = day_with_max_cases.first()['Date']

max_cases_value = df.agg(max('Total Confirmed cases').alias('max_cases')).first()['max_cases']
dates_with_max_cases = df.filter(col('Total Confirmed cases') == max_cases_value).select('Date')
dates_with_max_cases.show()  
max_cases_dates = dates_with_max_cases.collect()


# 3. The state with the second-largest number of COVID-19 cases
window = Window.orderBy(col('Total Confirmed cases').desc())
ranked_states = df.withColumn('rank', row_number().over(window))
# ranked_states.show() 
second_largest_state = ranked_states.filter(col('rank') == 2).select('Name of State / UT').first()

# 4. The Union Territory with the least number of deaths
least_deaths_ut = df.filter(col('Name of State / UT').isin(['delhi', 'puducherry', 'chandigarh', 'ladakh', 'jammu & kashmir'])) \
                   .groupBy('Name of State / UT') \
                   .agg(sum('Death').alias('total_deaths')) \
                   .orderBy('total_deaths')
# least_deaths_ut.show() 
least_deaths_ut_row = least_deaths_ut.first()

# 5. The state with the lowest Death to Total Confirmed cases ratio
ratio_df = df.withColumn('death_to_confirmed_ratio', col('Death') / col('Total Confirmed cases'))
# ratio_df.show() 
state_with_lowest_ratio = ratio_df.orderBy('death_to_confirmed_ratio').first()

# 6. The month with the most new recovered cases
df = df.withColumn('month', month('Date'))
monthly_recovered = df.groupBy('month').agg(sum('New recovered').alias('total_recovered'))
# monthly_recovered.show() 
month_with_max_recovered = monthly_recovered.orderBy(col('total_recovered').desc()).first()

# Convert month number to name
month_mapping = {
    1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
    7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'
}
month_name = month_mapping.get(month_with_max_recovered['month'], 'Unknown')

# Print results
print(f"Day with maximum Covid cases: {max_cases_dates}")
print(f"State with second-largest number of Covid cases: {second_largest_state['Name of State / UT']}")
print(f"Union Territory with the least number of deaths: {least_deaths_ut_row['Name of State / UT']}")
print(f"State with the lowest Death to Total Confirmed cases ratio: {state_with_lowest_ratio['Name of State / UT']}")
print(f"Month with the most new recovered cases: {month_name}")

+----------+
|      Date|
+----------+
|2020-08-06|
+----------+

Day with maximum Covid cases: [Row(Date=datetime.date(2020, 8, 6))]
State with second-largest number of Covid cases: maharashtra
Union Territory with the least number of deaths: ladakh
State with the lowest Death to Total Confirmed cases ratio: puducherry
Month with the most new recovered cases: July
